In [10]:
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader,
    StorageContext,
)
import os
from dotenv import load_dotenv
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.agent.workflow import AgentStream

load_dotenv()

True

### Vector Stores


In [ ]:
def load_or_create_index(vector_store_path: str, collection_name: str, create: bool, documents=None):
    # Initialize the ChromaDB client
    db = chromadb.PersistentClient(path=vector_store_path)

    # Create a new collection
    chroma_collection = db.get_or_create_collection(collection_name)

    # Assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    if create:
        # Create index
        index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    else:
        # Load index from storage
        index = VectorStoreIndex.from_vector_store(
            vector_store, storage_context=storage_context
        )
    return index


# Create a directory for the vector store if it doesn't exist
VECTOR_STORE_DIR = '../Quick-Examples/chroma_db'
if not os.path.exists(VECTOR_STORE_DIR):
    os.makedirs(VECTOR_STORE_DIR)
    # Load documents
    stoic_documents = SimpleDirectoryReader("../Quick-Examples/data/Pedagogia").load_data()
    taoist_documents = SimpleDirectoryReader("../Quick-Examples/data/Dir2").load_data()
    # Verify that there are no empty documents
    for doc1, doc2 in zip(stoic_documents, taoist_documents):
        if not doc1 or not doc1.text.strip():
            print("Documento vacío encontrado en Pedagogia")
        if not doc2 or not doc2.text.strip():
            print("Documento vacío encontrado en Dir2")

    # Create index
    stoic_index = load_or_create_index(
        documents=stoic_documents,
        vector_store_path=VECTOR_STORE_DIR, 
        collection_name='stoic_collection',
        create=True
    )
    taoist_index = load_or_create_index(
        documents=taoist_documents,
        vector_store_path=VECTOR_STORE_DIR, 
        collection_name='taoist_collection',
        create=True
    )
else:
    # Load index from storage
    stoic_index = load_or_create_index(
        vector_store_path=VECTOR_STORE_DIR, 
        collection_name='stoic_collection',
        create=False
    )
    taoist_index = load_or_create_index(
        vector_store_path=VECTOR_STORE_DIR, 
        collection_name='taoist_collection',
        create=False
    )

### Query Engine


In [3]:
from llama_index.llms.google_genai import GoogleGenAI

llm_gemini = GoogleGenAI(model="gemini-2.0-flash", api_key=os.environ["GEMINI_API_KEY"])

In [4]:
top_k = 5
response_mode = "tree_summarize" # Good for concise answers (summarization)

system_prompt = "Eres un maestro capaz de aconsejar y hablar de esta filosofía."
stoic_query_engine = stoic_index.as_query_engine(response_mode=response_mode, verbose=False, system_prompt=system_prompt, similarity_top_k=5, llm=llm_gemini)
taoist_query_engine = taoist_index.as_query_engine(response_mode=response_mode, verbose=False, system_prompt=system_prompt, similarity_top_k=5, llm=llm_gemini)

response = stoic_query_engine.query("Cual es el significado de la vida?")
print(response.response)
response = taoist_query_engine.query("Cual es el significado de la vida?")
print(response.response)

response.source_nodes[0]

La filosofía es lo que nos puede llevar a salvamento, y consiste en conservar el dios interior sin ultraje ni daño, para que triunfe de placeres y dolores, para que no obre al acaso, y se mantenga lejos de toda falsedad y disimulo, al margen de que se haga o no se haga esto o aquello; además, para que acepte la parte que le tocare en los varios sucesos accidentales e integrantes de su parte, como procedentes de aquel origen de quien procede él mismo; y, en particular, para que aguarde la muerte en actitud plácida, no viendo en ella otra cosa más que la disolución de los elementos de que consta todo ser viviente.

La vida es una salida y la muerte una entrada. El origen del universo es la madre de todas las cosas. Los seres nacen y crecen para retornar a su raíz. Lo blando y flexible pertenece a la vida.



NodeWithScore(node=TextNode(id_='54d02757-4879-4d0f-a7cc-56e262ed9c76', embedding=None, metadata={'page_label': '66', 'file_name': 'Lao_Tse_Tao-te-Ching.pdf', 'file_path': '/home/rprieto/RAG/Quick-Examples/../Quick-Examples/data/Dir2/Lao_Tse_Tao-te-Ching.pdf', 'file_type': 'application/pdf', 'file_size': 2047878, 'creation_date': '2025-04-24', 'last_modified_date': '2025-04-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f35bb195-9e96-4cf0-be64-674c9374d171', node_type='4', metadata={'page_label': '66', 'file_name': 'Lao_Tse_Tao-te-Ching.pdf', 'file_path': '/home/rprieto/RAG/Quick-Examples/../Quick-Examples/data/Dir2/Lao_Tse_Tao-te-Ching.pdf', 'file_type': 'application/pdf', 'file_size': 2047878, 'cre

### Function Tools


In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers.

    Parameters
    ----------
    a : float
        First number to be multiplied
    b : float
        Second number to be multiplied

    Returns
    -------
    float
        The product of the two numbers
    """
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum.
    
    Parameters
    ----------
    a : float
        First number to be added
    b : float
        Second number to be added

    Returns
    -------
    float
        The sum of the two numbers
    """
    return a + b

In [6]:
from llama_index.core.tools import FunctionTool, QueryEngineTool

# Wrap functions as tools
add_tool = FunctionTool.from_defaults(
    fn=add,
    name="add_tool",
    description="Add two numbers",
)

multiply_tool = FunctionTool.from_defaults(
    fn=multiply,
    name="multiply_tool",
    description="Multiply two numbers",
)

# Create a query engine tool for document retrieval
stoic_rag_tool = QueryEngineTool.from_defaults(
    query_engine=stoic_query_engine,
    name="stoic_rag_tool",
    description="A RAG engine that helps retrieve information about Marcus Aurelius and Stoicism",
)
taoist_rag_tool = QueryEngineTool.from_defaults(
    query_engine=taoist_query_engine,
    name="taoist_rag_tool",
    description="A RAG engine that helps retrieve information about Taoism and Lao Tzu",
)

In [11]:
from llama_index.core.agent import ReActAgent

# Create the agent with the tools using `from_tools`
agent = ReActAgent.from_tools(
    tools=[add_tool, multiply_tool, stoic_rag_tool, taoist_rag_tool],
    verbose=True,
    # max_iterations=5,
    # max_iterations_per_tool=3,
)

In [12]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
# Create the agent with the tools
agent = ReActAgent(
    tools=[add_tool, multiply_tool, stoic_rag_tool, taoist_rag_tool],
    llm=llm_gemini,
    system_prompt=(
        "You are an assistant expert in mathematics and philosophy. "
        "Use the provided tools to answer complex questions. "
        "If there are multiple mathematical operations, you must start first with multiplication."
    ),
)

ctx = Context(agent)

In [13]:
query = "Compare and contrast the idea of material things in Taoism and Stoicism." \
        "Also, what is 2+4?, and 25546345 * 345344?"

handler = agent.run(query, ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler
print("-" * 10)
print(response)
print("-" * 10)
response = await agent.run("What we were talking about?", ctx=ctx)
print("-" * 10)
print(response)
print("-" * 10)

Thought: The current language of the user is: English. I need to use the stoic_rag_tool and taoist_rag_tool to compare and contrast the idea of material things in Stoicism and Taoism. I also need to use the add_tool and multiply_tool to calculate 2+4 and 25546345 * 345344.
Action: stoic_rag_tool
Action Input: {"input": "Stoicism view on material possessions"}
```
Observation: Stoicism teaches that external things like possessions are indifferent. They are neither good nor bad in themselves. What matters is our virtue and how we use these things. We should be content with what we have and not be attached to material wealth.
```
Thought: The current language of the user is: English. I need to use the stoic_rag_tool and taoist_rag_tool to compare and contrast the idea of material things in Stoicism and Taoism. I also need to use the add_tool and multiply_tool to calculate 2+4 and 25546345 * 345344.
Action: taoist_rag_tool
Action Input: {"input": "Taoism view on material possessions"}
```
